In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
from fastai.tabular import *
!pip install category_encoders
from IPython import display
display.clear_output()

In [0]:
# import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
tdf = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

In [5]:
# df.info()
def dataessing(df, test):
  cat_col = ['Profession','Status','edu','Irregular','residence','prev_diagnosed','communication','Month','side_effects']
  num_col = ['age','day','Time','Doctor_visits','last_visit','cured_in']
  loan = ['Money']
  target = 'Y'
  train_DF = pd.DataFrame(); valid_DF = pd.DataFrame(); test_DF = pd.DataFrame();
  dfs = [df, test]
  for i in range(0,2):
    d = dfs[i]
    dfc = pd.DataFrame()
    dfc[cat_col] = d[cat_col].fillna("unknown")
    dfc[num_col+loan] = d[num_col+loan].fillna(0)
    dfc[num_col] = dfc[num_col].clip(lower=0)
    dfcd = dfc
    if i == 0:
      from sklearn.model_selection import train_test_split
      train_DF, valid_DF, y_train, y_test = train_test_split(dfcd, df[target], random_state=45, test_size=0.3)
      train_DF['Y'] = y_train
      valid_DF['Y'] = y_test
    else:
      test_DF = dfcd
  return train_DF, valid_DF, test_DF, num_col, cat_col, target

train_DF, valid_DF, test_DF, num_col, cat_col, target = dataessing(df, tdf)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
import category_encoders as ce
cb_enc = ce.CatBoostEncoder(cols=cat_col, random_state=7)
# Learn encoding from the training set
cb_enc.fit(train_DF[cat_col], train_DF['Y'])
# Apply encoding to the train and validation sets
train_DF[cat_col] = cb_enc.transform(train_DF[cat_col])
valid_DF[cat_col] = cb_enc.transform(valid_DF[cat_col])

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [10]:
train_DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19737 entries, 9089 to 23499
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Profession      19737 non-null  float64
 1   Status          19737 non-null  float64
 2   edu             19737 non-null  float64
 3   Irregular       19737 non-null  float64
 4   residence       19737 non-null  float64
 5   prev_diagnosed  19737 non-null  float64
 6   communication   19737 non-null  float64
 7   Month           19737 non-null  float64
 8   side_effects    19737 non-null  float64
 9   age             19737 non-null  float64
 10  day             19737 non-null  int64  
 11  Time            19737 non-null  int64  
 12  Doctor_visits   19737 non-null  int64  
 13  last_visit      19737 non-null  int64  
 14  cured_in        19737 non-null  int64  
 15  Money           19737 non-null  float64
 16  Y               19737 non-null  int64  
dtypes: float64(11), int64(6)
mem

In [15]:
traindata = train_DF
testdata = valid_DF.drop('Y',1)
num_col = train_DF.columns.drop('Y')
cat_col = []
dep_var = 'Y'
procs = [FillMissing, Categorify, Normalize]
data = (TabularList.from_df(traindata, procs=procs, cont_names=num_col, cat_names=cat_col)
          .split_by_idx(valid_idx=range(int(len(traindata)*0.9),len(traindata)))
          .label_from_df(cols=dep_var)
          .add_test(TabularList.from_df(testdata, cont_names=num_col, cat_names=cat_col, procs=procs))
          .databunch())

learn = tabular_learner(data, layers=[1000, 500], metrics=accuracy)
learn.fit_one_cycle(40)

epoch,train_loss,valid_loss,accuracy,time
0,0.483592,0.461926,0.858156,00:07
1,0.318322,0.271537,0.897163,00:07
2,0.268311,0.250047,0.898683,00:07
3,0.262514,0.240708,0.904255,00:08
4,0.249699,0.251883,0.894124,00:07
5,0.254761,0.236443,0.898683,00:08
6,0.244537,0.224665,0.907801,00:08
7,0.236766,0.234102,0.903242,00:07
8,0.233665,0.229074,0.905775,00:07
9,0.228520,0.221670,0.905268,00:08


In [14]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(accuracy_score(valid_DF['Y'], pred_class))
print(confusion_matrix(valid_DF['Y'], pred_class))
print(classification_report(valid_DF['Y'], pred_class))

0.8978602671710604
[[7175  316]
 [ 548  420]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      7491
           1       0.57      0.43      0.49       968

    accuracy                           0.90      8459
   macro avg       0.75      0.70      0.72      8459
weighted avg       0.89      0.90      0.89      8459



In [0]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)

le_y = le.inverse_transform(pred_class)

output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')